<a href="https://colab.research.google.com/github/isayahc/google_colab/blob/main/chapgpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

import torch
from torch import nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config

class ChatGPT2(nn.Module):
    def __init__(self, config):
        super(ChatGPT2, self).__init__()
        self.transformer = GPT2LMHeadModel(config)
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    def forward(self, input_ids, past=None):
        transformer_outputs = self.transformer(input_ids, past=past)
        hidden_states = transformer_outputs[0]
        return hidden_states, transformer_outputs[1]

    def generate_response(self, conversation_context, max_length):
        # Convert conversation_context to input_ids
        input_ids = self.tokenizer.encode(conversation_context, return_tensors='pt')
        # Generate response
        generated = self.transformer.generate(input_ids, max_length=max_length)
        # Convert response tokens to string
        response = self.tokenizer.decode(generated[:, input_ids.shape[-1]:][0], skip_special_tokens=True)
        return response

# Initialize the model configuration
config = GPT2Config.from_pretrained('gpt2')

# Initialize the model
model = ChatGPT2(config)

# Now, we can use the model for inference
conversation_context = "Hello, how are you?"
max_length = 50

response = model.generate_response(conversation_context, max_length)
print(response)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.4 MB/s eta 0:00:00


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 duty duty dutycollege profitability redirect redirect redirect redirect instr resembled funnel Spicereksekseksekseksekseksekseks illusions illusions involve Declaration spellingzipzipzip Jub Jub Jub Jub Cloud HEADarah Buddha Buddha Buddha Buddha gastro gastro gastro
